# The Voting Machine

The Voting Machine is a system that is supposed to provide a basis for free, independant and safe elections to groups of people 
like political parties, companies, organisations and even whole governments.

## Elections
Elections are the basis of modern democracy. This implies that there is always a party involved who would benefit of breaking the rules, either by manipulating the outcome, ability of blackmailing and corrupting people or only by interrupting and disturbing the election.
Thus there are very high demands on the concept and later on the implementation of the application. Currently I am not striving for a 100% safe and tested application, but a proof of concept that will demonstrate that it is possible to have electronic elections that can reach the standards of manual box-voting.

Following I will elaborate on the concept and how the different parts work and link together.

## The Idea

The application consists of three parts: **boxserver**, **idserver** and **userclient**. This setup makes it possible to split powers, respectively responsibilites. As said before, one has to assume that there always may be a party involved who has interest in corrupting the system. However, splitting the responsibilities - not only technically but also geographically and with different administrators - 
can reduce the chance that one corrupted element is able to have a large impact on the whole, and even if so, then not unnoticed.

### Identification
At first it is necessary to make sure that only someone who is registered as legitimate voter can actually vote, and if so, 
only with one counted vote. This means that a natural person has to identify in the system in any given way. Nowadays many different systems are tested and (more or less) successfully run. For the election system here being discussed it is not necessary to elaborate on the many ways of identification. However, one should note that a change in medium has to take place in order to improve security.
As a preliminary solution a bank-like identification system is implemented which works like that:

1. To all voters, two letters are sent via secured measures.The first letter contains a password, the second contains a two-digit number. I will call the password (string) P, the number (cast to string) N. To identify, you will connect to the idserver where two (salted) hashes are saved for every voter: (P) and (P+N).

2. The user enters both, P and N. If P and N are correct, you can vote as described later without problems. If only P is correct, but not N, you can vote as if everything is correct - but boxserver will not count the vote in the end, without it being known who submitted the false vote. This measure enables people under pressure to hand over their password but without giving away their "real" voice.

### IDServer
The idserver ONLY handles the list that determines who can and who cannot vote, being called register. Identification as described above may be a function of the idserver but also could be provided by external means. The register is a dictionary-type (hashtable) by name. It contains three boolean fields: identified, voting, voted and may also contain the (P) and (P+N) hashes as described above.
Whichever way a voter has been identified, the field is set to identified. A random number is generated which is called ballotnum.
The ballotnum is stored in the currentVoters list, together with the corresponding name. A secure connection is established between idserver and boxserver and the ballotnum is transfered (NOT the name or anything that may give away the identity of the voter). The ballotnum is also given to the userclient.

### BoxServer

After having identified and gotten a ballotnum from the idserver, the userclient connects to the boxserver and provides the ballotnum log in.
The boxserver checks if the ballotnum has been given by the idserver and allows the user to vote if this is the case.


1. listmaker: produces the list of parties and voters, pickled. These lists can be distributed encrypted and with md5 hash, as needed, independantly from the servers.
2. idserver: loads voterlist and lets voters log in the system by password. Transfers ballot and action number to boxservers and voters.
3. boxserver: loads partylist and lets voters vote, with their ballot and action number.
4. userclient: gets password fed by user (which arrived by mail/mobile phone/..), logs into idserver and gets the ballot and action num, votes with these on boxserver.


This file explains how the VotingMachine is to be used.

1) Run the listmaker to create two lists: "register" and "partylist". The partylist requires to enter a name and description for each party. The register requires name, birthdate and postal code as base data for every registered voter. With every entry, the listmaker returns the password for the entered voter that is NOT saved by the system but has to be processed seperately. Ideally the password is parsed together with the name and automatically inserted into a serial letter.
The register file has to be sent securely to the idserver, the partylist to the boxservers.

2) Start the idserver. It will run on port 10001.
On startup of the idserver you need to entered the location of the register file, then the idserver will create a one-time key for this session. You need to enter a valid location where the public key file should be saved to.
After the key is created and saved you have to make it available to the operators of the boxservers.


3) Now the boxservers need to be fired up. They will run on port 10002.
Every operator of a boxserver needs available the partylist and the public key file and also the IP address of idserver.

4) Finally userclients can be used for voting. Userclients run on port 10003.
For this, the user has to know the IP address of the idserver, the data with which he registered and the associated password he got (via mail for instance).
The client will first connect to the idserver, get the public key and send the login data as well as a temporary public key, both encrypted by the public key of the idserver.
This will identify the user in the system. The idserver returns two numbers to the userclient: an actionNum and a ballotNum. Both also are sent securely to the boxserver where the user will vote on.
The userclient then connects to the boxserver of the users choice. The boxserver then provides the client with the partylist and his public key. The client gives the two numbers to log in, encrypted. After the boxserver verified the login, the client sends his vote to the boxserver.

At this point the vote is fixed and in most cases the user is done with the procedure.
However, there is the possibility to log into the system another time to revise the vote. This can be done after a 3 minute-delay, as often as required.

5) After the election period has ended, the register file on the idserver and the voteLists on the boxservers are published and count.

TODO:

    1) Need to reimplement listmaker to use MySQL. [MySQLdb is corrupted! :-( ]
    2) Adapt idserver to use the MySQL.  [MySQLdb is corrupted! :-( ]
    3) Implement a GUI in userclient.
    4) Implement a network interface between all 3 parts.
    5) Implement an html interface for voteserver that auto-publishes the results.